This notebook contains the Bias correction over precipitation with the method BcsdPrecipitation(return_anoms=False)

In [ ]:
# import modules and functions

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# import data
from Functions_ImportData import import_treat_modeled_NEX_GDDP_CMIP6
from Functions_ImportData import import_filtered_NOAA_obs
from Functions_ImportData import import_treat_obs_NOAA
from Functions_ImportData import import_treat_modeled_NEX_GDDP_CMIP6_close_to_stationNOAA
from Bias_correction_function import BC
from Bias_correction_function import treat_data_for_test
from Bias_correction_function import BCSD_Precipitation_return_anoms_to_apply
from Functions_Indicators import add_year_month_season

In [ ]:
climate_var = 'pr' # 'tas'
unit='mm_per_day'
resolution = 'day'

start_y = 2030
stop_y = 2065
# precipitation : 'pr'

name_station = 'BEIRA, MZ'
name_project = 'WTP_Mutua_EIB'

In [ ]:
# import observation data
data_obs_NOAA_filtered=import_filtered_NOAA_obs()

In [ ]:
# import historic modeled data
climate_var_NEX_GDDP_CMIP6_EmplacementStation=import_treat_modeled_NEX_GDDP_CMIP6_close_to_stationNOAA(climate_var, 'mm_per_day')

In [ ]:
# import data to correct
climate_var_NEX_GDDP_CMIP6_Project=import_treat_modeled_NEX_GDDP_CMIP6(climate_var,'mm_per_day','day',1950,2100)

In [ ]:
climate_var_NEX_GDDP_CMIP6_Project

In [ ]:
climate_var_NEX_GDDP_CMIP6_Project_to_correct=climate_var_NEX_GDDP_CMIP6_Project[climate_var_NEX_GDDP_CMIP6_Project['Year'].between(start_y,stop_y)]

In [ ]:
list_model=list(set(climate_var_NEX_GDDP_CMIP6_Project_to_correct['Model']))#.remove('NESM3')
list_model.remove('NESM3')

In [ ]:
df_pr_bc_corrected=pd.DataFrame()
for name_project in list(set(climate_var_NEX_GDDP_CMIP6_Project_to_correct['Name project'])):
    climate_var_NEX_GDDP_CMIP6_Project_temp=climate_var_NEX_GDDP_CMIP6_Project_to_correct[climate_var_NEX_GDDP_CMIP6_Project_to_correct['Name project']==name_project]
    for scenario in list(set(climate_var_NEX_GDDP_CMIP6_Project_to_correct['Experiment'])):
        climate_var_NEX_GDDP_CMIP6_Project_temp_2=climate_var_NEX_GDDP_CMIP6_Project_temp[climate_var_NEX_GDDP_CMIP6_Project_temp['Experiment']==scenario]
        for model in list_model:
            print('For '+name_project+', '+scenario+' and '+model)
            climate_var_NEX_GDDP_CMIP6_Project_temp_3=climate_var_NEX_GDDP_CMIP6_Project_temp_2[climate_var_NEX_GDDP_CMIP6_Project_temp_2['Model']==model]
            df_pr = treat_data_for_test(data_obs_NOAA_filtered,'PRCP',climate_var_NEX_GDDP_CMIP6_EmplacementStation,'Mean of the daily precipitation rate mm/day','BEIRA, MZ',model)
            (X_pcp,y_pcp,out)=BCSD_Precipitation_return_anoms_to_apply(df_pr,climate_var_NEX_GDDP_CMIP6_Project_temp_3[['Date','Mean of the daily precipitation rate mm_per_day']].dropna().reset_index(drop=True))
            out = out.reset_index()
            out['Name project']=name_project
            out['Experiment']=scenario
            out['Model']=model
            df_pr_bc_corrected=pd.concat([df_pr_bc_corrected,out])

In [ ]:
#df_pr_bc_corrected[['Date']].iloc[1000][0].strftime('%Y-%m-%d')

In [ ]:
#df_pr_bc_corrected[['Date']].iloc[1000][0]

In [ ]:
#len(df_pr_bc_corrected[['Date']])

In [ ]:
# convert date in str format

In [ ]:
for date in np.arange(0,len(df_pr_bc_corrected[['Date']])):
    df_pr_bc_corrected[['Date']].iloc[date][0]=df_pr_bc_corrected[['Date']].iloc[date][0].strftime('%Y-%m-%d')

In [ ]:
df_pr_bc_corrected = add_year_month_season(df_pr_bc_corrected,'Date')

In [ ]:
df_pr_bc_corrected

In [ ]:
path = r'\\COWI.net\projects\A245000\A248363\CRVA\Datasets\NEX-GDDP-CMIP6-AllMoz\csv_file'

In [ ]:
df_pr_bc_corrected.to_csv(os.path.join(path,climate_var,climate_var+'_'+unit+'_'+resolution+'_'+str(start_y)+'-'+str(stop_y)+'_BiasCorrected',climate_var+'_'+'mm_per_day'+resolution+str(start_y)+'-'+str(stop_y)+'_BiasCorrected.csv'))